## Integration of functions loaded in MPS

This notebook shows how to integrate functions loaded in MPS by contracting them with predefined quadrature MPS. These quadrature MPS can be straightforwardly generalized to multivariate scenarios by means of tensor products in both qubit orders, either serial or interleaved. Moreover, this integration procedure avoids the *curse of dimensionality*, as it has a cost that scales polynomially with the dimension of the function instead than exponentially.

The main prerequisite for this *quantum-inspired* integration method is to dispose of a prior multivariate function loaded in MPS. This can be done following several methods, such as MPS orthogonal-polynomial expansions (see the `chebyshev_composition.ipynb` example), tensor cross-interpolation (TCI, see the `tt-cross.ipynb` example), multiscale interpolative constructions, solutions to partial differential equations, etc.

For this example, we are going to load a trivially simple multivariate function 

$f(x_1, \ldots, x_{10}) = \prod_{i=1}^{10} x_i^3$.

The integral of this function in $\Omega = [-1, 1] \times \ldots \times [-1, 1]$ is naively zero. Even though it can be analytically constructed, we use tensor cross-interpolation as it generalizes to a wider range of functions. Then, we integrate it following a Clenshaw-Curtis quadrature rule.

In [ ]:
import numpy as np

from seemps.analysis.mesh import ChebyshevInterval, Mesh, mps_to_mesh_matrix
from seemps.analysis.cross import BlackBoxLoadMPS, cross_dmrg
import seemps.tools

seemps.tools.DEBUG = 2

start, stop = -1, 1
num_qubits = 3
interval = ChebyshevInterval(
    start, stop, 2**num_qubits, endpoints=True
)  # Chebyshev extrema

dimension = 10
mesh = Mesh([interval] * dimension)

func = lambda tensor: np.sum(tensor**3, axis=0)  # x^3 * y^3 * ...
map_matrix = mps_to_mesh_matrix([num_qubits] * dimension)
physical_dimensions = [2] * num_qubits * dimension
black_box = BlackBoxLoadMPS(func, mesh, map_matrix, physical_dimensions)

mps_func = cross_dmrg(black_box).mps

Next, we construct the quadrature MPS. We can integrate the function directly using the auxiliar routine `integrate_mps`, but in this example we construct the quadrature manually.

In [ ]:
from seemps.analysis.integration import mps_clenshaw_curtis
from seemps.analysis.factories import mps_tensor_product

mps_quad_1d = mps_clenshaw_curtis(start, stop, num_qubits)
mps_quad_10d = mps_tensor_product([mps_quad_1d] * dimension)

Finally, integrate the MPS by taking the scalar product `scprod` of the function MPS with the quadrature MPS.

In [ ]:
from seemps.state import scprod

integral_exact = 0
integral_mps = scprod(mps_func, mps_quad_10d)

print("Integration error: ", np.max(np.abs(integral_exact - integral_mps)))

More complicated quadrature rules, such as those arising from non-implemented quadrature rules or non-binary MPS structures, can be loaded in MPS form using the function `quadrature_mesh_to_mps`, which employs tensor cross-interpolation to load it in MPS. In practice, TCI proves to be accurate and robust for encoding these types of MPS in most situations.

For example, let us integrate the same function but this time encoded as an MPS using a ternary base. Now, we consider 2 qutrits instead of 3 for each dimension, such that it has $3^2 = 9$ nodes instead of the previous $2^3 = 8$. This example illustrates the generality of TCI and MPS-based integration.

In [ ]:
from seemps.analysis.integration import mesh_to_quadrature_mesh, quadrature_mesh_to_mps

num_qutrits = 2
interval = ChebyshevInterval(start, stop, 3**num_qutrits)
mesh = Mesh([interval] * dimension)

map_matrix = mps_to_mesh_matrix([num_qutrits] * dimension, base=3)
physical_dimensions = [3] * num_qutrits * dimension
black_box = BlackBoxLoadMPS(func, mesh, map_matrix, physical_dimensions)

mps_func_3 = cross_dmrg(black_box).mps
print(mps_func_3.physical_dimensions())

quad_mesh = mesh_to_quadrature_mesh(mesh)
mps_quad_3 = quadrature_mesh_to_mps(quad_mesh, map_matrix, physical_dimensions)
print(mps_quad_3.physical_dimensions())

integral_exact = 0
integral_mps = scprod(mps_func_3, mps_quad_3)
print("Integration error: ", np.max(np.abs(integral_exact - integral_mps)))